In [23]:
import numpy as np
import whisper, torch
from pathlib import Path
import jiwer
from jiwer import *
import csv
import unicodedata
import matplotlib.pyplot as plt
import math
import pandas as pd


Normalizar los textos

In [24]:
class RemoveDiacritics:
    def __call__(self, x):
        if isinstance(x, str):
            # Normaliza a NFD y elimina los caracteres de acento (categoria Mn)
            return "".join(
                c for c in unicodedata.normalize("NFD", x)
                if unicodedata.category(c) != "Mn"
            )
        elif isinstance(x, list):
            # Procesar recursivamente cada elemento
            return [self.__call__(s) for s in x]
        else:
            return x
tr_w = jiwer.Compose([
    Strip(),
    ToLowerCase(),   
    RemoveDiacritics(),           
    RemovePunctuation(),               
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(),
])

tr_c = jiwer.Compose([
    Strip(),
    ToLowerCase(),
    RemoveDiacritics(),
    RemovePunctuation(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfChars(),
])

tr= jiwer.Compose([
    Strip(),
    ToLowerCase(),   
    RemoveDiacritics(),           
    RemovePunctuation(),               
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
])

Cálculo de métricas

In [25]:
def calcular_metricas(refs:str, hyps: str):
    #DADA UNA REFERENCIA (transcripción real) Y UNA HIPÓTESIS (output transcripto por el modelo) para un cierto MODELO, devuelve un diccionario de métricas asociadas a ese audio y transcripción
    metricas = dict()

    # Word Error Rate
    out_wer = jiwer.process_words(refs, hyps, reference_transform=tr_w, hypothesis_transform=tr_w)
    # Character Error Rate
    out_cer = jiwer.process_characters(refs, hyps, reference_transform=tr_c, hypothesis_transform=tr_c)

    metricas["WER"] = out_wer.wer
    metricas["CER"] = out_cer.cer
    metricas["ALINEACION"] = jiwer.visualize_alignment(out_wer)
    metricas["ERRORES"] = jiwer.visualize_error_counts(out_wer)

    return metricas

Cálculo del promedio de las métricas a partir de un diccionario con todos nuestros resultados

In [26]:
def metricas_promedio(models_data:dict[dict[dict[str:str]]]):
    #DEVUELVE: un diccionario donde las claves son los modelos y el valor es el WER promedio asociado a ese modelo
    promediosWER = dict()
    promediosCER = dict()
    for modelo in models_data.keys():
        sumaWER = 0
        sumaCER = 0
        for registro in models_data[modelo].keys():
                sumaWER += models_data[modelo][registro]["METRICS"]["WER"]
                sumaCER += models_data[modelo][registro]["METRICS"]["CER"]
        promedioWER = sumaWER/len(models_data[modelo])
        promedioCER = sumaCER/len(models_data[modelo])
        promediosWER[modelo] = promedioWER
        promediosCER[modelo] = promedioCER
        
    return promediosWER, promediosCER

Confección del diccionario 

In [27]:
def eval(data_archivo:str, modelo:str, models_data):
    print("MODELO: ", modelo)

    if modelo not in models_data:
        models_data[modelo] = {}

    model = whisper.load_model(modelo)

    with open(data_archivo, encoding="utf-8") as f:
        r = csv.DictReader(f, delimiter=",")
        for registro in r: 
            audio = registro['AUDIO']
            ref = registro["TRANSCRIPCION"]

            hyp = model.transcribe(audio, language="es", fp16=False)["text"]

            metricas = calcular_metricas(ref, hyp)
            
            audio_key = Path(audio).stem    
            print(audio_key)
        
            models_data[modelo][audio_key] = dict()
            models_data[modelo][audio_key]["REF"] = ref
            models_data[modelo][audio_key]["HYP"] = hyp
            models_data[modelo][audio_key]["METRICS"] = dict()
            models_data[modelo][audio_key]["METRICS"]["WER"] = metricas["WER"]
            models_data[modelo][audio_key]["METRICS"]["CER"] = metricas["CER"]
            models_data[modelo][audio_key]["METRICS"]["ALINEACION"] = metricas["ALINEACION"]
            models_data[modelo][audio_key]["METRICS"]["ERRORES"] = metricas["ERRORES"]

Función para visualizar el diccionario

In [28]:
def printear_models_data(models_data: dict[dict[dict[str:str]]]):
    print("\nRESULTADOS: \n")
    for modelo, registros in models_data.items():
        print(f"MODELO: {modelo}\n")
        for registro, info in registros.items():
            print(f"    {registro}")
            print("         REF:")
            print(f"             {info['REF']}")
            print("         HYP:")
            print(f"            {info['HYP']}")

            m = info["METRICS"]
            print("         MÉTRICAS:")
            print(f"            - WER: {m['WER']:.3f}")
            print(f"            - CER: {m['CER']:.3f}")
            print("            - ALINEACIÓN (palabras):")
            print(m["ALINEACION"])
            print("            - ERRORES (resumen):")
            print(m["ERRORES"])
            print("\n")
        print("\n\n")



Graficos de las métricas promedio

In [29]:
def plot_incremental(
    metrica:str,
    models_data,
    model_order=None,
    sort_audios="alphabetical",
    figsize=(12, 5),
    save_prefix="wers_step",
):
    """
    Grafica WER por audio de manera incremental:
    - Primer gráfico: solo el primer modelo
    - Segundo: primer modelo + segundo modelo
    - etc.
    """

    # audios: union de todos los keys
    audios = sorted({a for registros in models_data.values() for a in registros.keys()})

    # usar orden alfabético de modelos si no se pasa order explícito
    if model_order is None:
        model_order = sorted(models_data.keys())

    # preparar datos
    wers_by_model = {}
    for m in model_order:
        wers_by_model[m] = [
            models_data[m][a]["METRICS"][metrica] if a in models_data[m] else None
            for a in audios
        ]

    saved_paths = []
    for k in range(1, len(model_order) + 1):
        plt.figure(figsize=figsize)
        x = list(range(len(audios)))
        for m in model_order[:k]:
            y = [
                float("nan") if v is None else v
                for v in wers_by_model[m]
            ]
            plt.scatter(x, y, label=m)
        etiquetas = [a[:6] + a[9] for a in audios]
        plt.xticks(x, etiquetas, rotation=90)
        plt.xlabel("Audio")
        plt.ylabel(metrica)
        plt.title(f"{metrica} por audio — modelos: {', '.join(model_order[:k])}")
        plt.legend()
        plt.tight_layout()

        path = f"{save_prefix}_{metrica.lower()}_{k}.png"
        plt.savefig(path, dpi=150, bbox_inches="tight")
        saved_paths.append(path)
        plt.close()  # no abrir ventana en scripts


    return saved_paths

In [30]:
def plot_boxplots_metric(models_data: dict,
                         metric: str,
                         model_order=None,
                         figsize=(10, 5),
                         save_path=None,
                         show=False):
    """
    Dibuja un gráfico con N boxplots (uno por modelo) para la métrica indicada.
      - metric: "WER" o "CER" (insensible a mayúsculas)
      - model_order: lista explícita con el orden de modelos (si None, usa orden alfabético)
      - figsize: tamaño de la figura en pulgadas
      - save_path: ruta del PNG a guardar (si None, genera un nombre a partir de la métrica)
      - show: si True, llama a plt.show() (útil en entornos con GUI)
    Devuelve la ruta del archivo guardado.
    """
    metric_key = metric.strip().upper()
    if metric_key not in {"WER", "CER"}:
        raise ValueError("metric debe ser 'WER' o 'CER'")

    # Orden de modelos
    if model_order is None:
        model_order = sorted(models_data.keys())

    # Recolectar datos por modelo
    data_by_model = []
    labels = []
    for m in model_order:
        vals = []
        for _, info in models_data.get(m, {}).items():
            v = info.get("METRICS", {}).get(metric_key, None)
            if v is None:
                continue
            try:
                v = float(v)
            except Exception:
                continue
            if not (math.isnan(v) or math.isinf(v)):
                vals.append(v)
        if len(vals) > 0:
            data_by_model.append(vals)
            labels.append(m)

    if not data_by_model:
        raise ValueError(f"No hay datos para la métrica {metric_key} en models_data.")

    # Graficar
    plt.figure(figsize=figsize)
    plt.boxplot(data_by_model, showfliers=True)
    plt.xticks(range(1, len(labels) + 1), labels, rotation=45, ha="right")
    plt.ylabel(metric_key)
    plt.title(f"{metric_key} por modelo (boxplot)")
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()

    # Guardar y opcionalmente mostrar
    if save_path is None:
        save_path = f"boxplot_{metric_key.lower()}.png"
    plt.savefig(save_path, dpi=150, bbox_inches="tight")
    if show:
        plt.show()
    plt.close()

    return save_path

Llevar todo a un excel

In [31]:
def exportar_excel(models_data, path="resultados.xlsx"):
    """
    Exporta a un Excel con columnas: audio, REF, HYP
    """
    filas = []
    for modelo, audio in models_data.items():
        for audio_key, info in audio.items():
            filas.append({
                "modelo": modelo,
                "audio": audio_key,
                "REF": tr(info.get("REF", "")),
                "HYP": tr(info.get("HYP", ""))
            })
    df = pd.DataFrame(filas)
    df.to_excel(path, index=False)
    print(f"Archivo Excel guardado en {path}")
    

In [32]:
models_data = dict()
eval("data_adultos.csv","tiny", models_data)
eval("data_adultos.csv","base", models_data)
eval("data_adultos.csv","small", models_data)
eval("data_adultos.csv","medium", models_data)
eval("data_adultos.csv","large", models_data)
eval("data_adultos.csv","turbo", models_data)

printear_models_data(models_data)

WER_promedio, CER_promedio = metricas_promedio(models_data) #WER_promedio es un diccionario donde guardamos para cada modelo, su WER_promedio, idem para CER_promedio
    
print("\nWER PROMEDIO PARA CADA MODELO:")
print(WER_promedio)
print("\nCER PROMEDIO PARA CADA MODELO:")
print(CER_promedio)

paths = plot_incremental(
    "WER",
    models_data,
    model_order=["tiny","base","small","medium","large","turbo"],
    sort_audios="first_model"
)

paths = plot_incremental(
    "CER",
    models_data,
    model_order=["tiny","base","small","medium","large","turbo"],
    sort_audios="first_model"
)
print("Gráficos guardados en:")
for p in paths:
    print("   ", p)

orden = ["tiny","base","small","medium","large","turbo"]  # ajustá según tus modelos
png_wer = plot_boxplots_metric(models_data, "WER", model_order=orden, figsize=(12,5))
png_cer = plot_boxplots_metric(models_data, "CER", model_order=orden, figsize=(12,5))

print("Guardados en:", png_wer, "y", png_cer)

exportar_excel(models_data, "transcripciones.xlsx")

MODELO:  tiny


RuntimeError: Numpy is not available